In [4]:
import os
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

def prepare_and_augment_data(source_directory):
    # Initialize the ImageDataGenerator with augmentation parameters
    # Create the target directories inside the source directory
    train_directory = os.path.join(source_directory, "train")
    test_directory = os.path.join(source_directory, "test")
    os.makedirs(train_directory, exist_ok=True)
    os.makedirs(test_directory, exist_ok=True)
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    # Create target directories if they don't exist
    os.makedirs(train_directory, exist_ok=True)
    os.makedirs(test_directory, exist_ok=True)

    # Loop through each file in the source directory
    for filename in os.listdir(source_directory):
        if filename.endswith(".jpg"):
            # Extract the student name from the filename
            student_name = os.path.splitext(filename)[0]
            
            # Create directories for training and testing for each student
            student_train_directory = os.path.join(train_directory, student_name)
            student_test_directory = os.path.join(test_directory, student_name)
            os.makedirs(student_train_directory, exist_ok=True)
            os.makedirs(student_test_directory, exist_ok=True)

            # Load the image
            img_path = os.path.join(source_directory, filename)
            img = load_img(img_path)
            x = img_to_array(img)
            x = x.reshape((1,) + x.shape)  # Reshape for the ImageDataGenerator

            # Save the original image to the training directory
            shutil.copy(img_path, os.path.join(student_train_directory, filename))

            # Generate and save augmented images
            i = 0
            for batch in datagen.flow(x, batch_size=1, save_to_dir=student_test_directory, save_prefix=student_name, save_format='jpg'):
                i += 1
                if i >= 5:  # Generate 5 augmented images for testing
                    break

            # Ensure we have 10 images in training directory (including the original)
            num_existing_train_images = len([f for f in os.listdir(student_train_directory) if f.endswith('.jpg')])
            if num_existing_train_images < 10:
                # Additional augmented images for training
                for batch in datagen.flow(x, batch_size=1, save_to_dir=student_train_directory, save_prefix=student_name, save_format='jpg'):
                    num_existing_train_images += 1
                    if num_existing_train_images >= 10:
                        break

    print("Data augmentation and organization completed.")



In [88]:

# Example usage
source_directory = "D:\SSearch project\Clash of Titans\sample images folder 2"

prepare_and_augment_data(source_directory)

<>:2: SyntaxWarning: invalid escape sequence '\S'
<>:2: SyntaxWarning: invalid escape sequence '\S'
C:\Users\ASUS\AppData\Local\Temp\ipykernel_21884\31739658.py:2: SyntaxWarning: invalid escape sequence '\S'
  source_directory = "D:\SSearch project\Clash of Titans\sample images folder 2"


Data augmentation and organization completed.


# Face extraction

In [3]:
import os
import numpy as np
import cv2 as cv
from mtcnn.mtcnn import MTCNN

class FACELOADING:
    def __init__(self, directory):
        self.directory = directory
        self.target_size = (160, 160)
        self.X = []
        self.Y = []
        self.detector = MTCNN()

    def extract_face(self, filename):
        img = cv.imread(filename)
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        detected_faces = self.detector.detect_faces(img)
        if detected_faces:
            x, y, w, h = detected_faces[0]['box']
            x, y = abs(x), abs(y)
            face = img[y:y+h, x:x+w]
            face_arr = cv.resize(face, self.target_size)
            return face_arr
        return None

    def load_faces(self, dir):
        FACES = []
        for im_name in os.listdir(dir):
            if im_name.endswith(".jpg"):
                path = os.path.join(dir, im_name)
                face = self.extract_face(path)
                if face is not None:
                    FACES.append(face)
        return FACES

    def load_classes(self):
        for sub_dir in os.listdir(self.directory):
            path = os.path.join(self.directory, sub_dir)
            if os.path.isdir(path):
                FACES = self.load_faces(path)
                labels = [sub_dir] * len(FACES)
                self.X.extend(FACES)
                self.Y.extend(labels)
        return np.asarray(self.X), np.asarray(self.Y)





In [2]:
# Example usage:
def prepare_and_load_faces(source_directory, augmentations=10):
    prepare_and_augment_data(source_directory)
    faceloading_train = FACELOADING(os.path.join(source_directory, "train"))
    faceloading_test = FACELOADING(os.path.join(source_directory, "test"))
    
    X_train, Y_train = faceloading_train.load_classes()
    X_test, Y_test = faceloading_test.load_classes()
    return X_train, X_test, Y_train, Y_test


In [5]:
# Example usage
source_directory = "D:\SSearch project\Clash of Titans\sample images folder 2"

X_train, X_test, Y_train, Y_test = prepare_and_load_faces(source_directory)


<>:2: SyntaxWarning: invalid escape sequence '\S'
<>:2: SyntaxWarning: invalid escape sequence '\S'
C:\Users\ASUS\AppData\Local\Temp\ipykernel_20992\2558219617.py:2: SyntaxWarning: invalid escape sequence '\S'
  source_directory = "D:\SSearch project\Clash of Titans\sample images folder 2"


Data augmentation and organization completed.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


In [6]:
len(X_test)

50

In [7]:
len(Y_test)

50

In [106]:
from keras_facenet import FaceNet
embedder = FaceNet()

def get_embedding(face_img):
    face_img = face_img.astype('float32') # 3D(160x160x3)
    face_img = np.expand_dims(face_img, axis=0)
    # 4D (Nonex160x160x3)
    yhat= embedder.embeddings(face_img)

    return yhat[0] # 512D image (1x1x512)

In [107]:
EMBEDDED_X_Train = []
EMBEDDED_X_Test = []
count = 0
for img in X_train:
    EMBEDDED_X_Train.append(get_embedding(img))
    count +=1
    print(count)
count = 0
for img in X_test:
    EMBEDDED_X_Test.append(get_embedding(img))
    count +=1
    print(count)

EMBEDDED_X_Train = np.asarray(EMBEDDED_X_Train)
EMBEDDED_X_Test = np.asarray(EMBEDDED_X_Test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
6
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
8
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
9
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
11
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
12
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
13
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
14
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
15
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
16
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
17
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
18
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
19
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
21
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
22
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
23
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
24
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s

In [109]:
from sklearn.preprocessing import LabelEncoder

encoder_train = LabelEncoder()
encoder_train.fit(Y_train)
Y_train = encoder_train.transform(Y_train)

In [110]:
encoder_test = LabelEncoder()
encoder_test.fit(Y_test)
Y_test = encoder_train.transform(Y_test)

In [108]:
print(len(EMBEDDED_X_Train))
print(len(EMBEDDED_X_Test))

100
100


In [8]:
import numpy as np

unique_labels = np.unique(Y_test)
print(f"Unique labels in Y_train: {unique_labels}")

Unique labels in Y_train: ['Aakash Gupta_231040401_HALL14_335'
 'Aakashdeep Singh_231110001_HALL14_233'
 'Aayush Chaudhari_231040001_HALL14_399'
 'Abhay Sharma_231030001_HALL14_190' 'Abhay_231050001_HALL14_563'
 'Abhijeet Gangwar_231010001_HALL14_17' 'Abhijeet_231030002_HALL14_238'
 'Abhinandan Singh Baghel_231110002_HALL14_224'
 'Abhinav Pratap Singh_231140001_HALL14_299'
 'Abhinav Saumya_231050003_HALL14_546']


In [112]:
import numpy as np
import hashlib

def hash_image(image):
    # Convert image to a hashable format (e.g., bytes)
    image_bytes = image.tobytes()
    return hashlib.md5(image_bytes).hexdigest()

def unique_images(X_test):
    hashes = np.array([hash_image(image) for image in X_test])
    unique_hashes = np.unique(hashes)
    return unique_hashes

unique_image_hashes = unique_images(X_test)
print(f"Number of unique images in X_train: {len(unique_image_hashes)}")


Number of unique images in X_train: 100


# Model

In [114]:
from sklearn.svm import SVC
model = SVC(kernel='linear', probability=True)
model.fit(EMBEDDED_X_Train, Y_train)

SVC(kernel='linear', probability=True)

In [115]:
from sklearn.metrics import accuracy_score

ypreds_test = model.predict(EMBEDDED_X_Test)


accuracy_score(Y_test, ypreds_test)

1.0

In [116]:
ypreds_train = model.predict(EMBEDDED_X_Train)
accuracy_score(Y_train, ypreds_train)

1.0